# Data preparation notebook

This notebook shows how a sample query is prepared from data, given as an input.

## Setup

`IMPORTANT`: Import original "Barbora Homework  ML Engineer.xlsx" into working directory and rename to `"barbora.xlsx"`. 




In [ ]:
!pip3 install PrettyTable
!pip3 install statsmodels
!pip3 install openpyxl

In [28]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from datetime import datetime, date, time

from prettytable import PrettyTable
from statsmodels.tsa.seasonal import seasonal_decompose

## Data ingestion

Any 30 days data given in a format mathching of revenue (30 rows) and weather (between 30 days) would work. For instance, uncomment following lines.

In [ ]:
# # Store 30 days sample data


# df = pd.read_excel('barbora.xlsx', sheet_name="Weather")
# df[df['dt'].between(20180101, 20180130)].to_csv('example_weather.csv', index=False)

# df1 = pd.read_excel("barbora.xlsx", sheet_name="Revenue")
# df1[:30].to_csv('example_revenue.csv', index=False)

Ingest sample query data

In [60]:

df1 = pd.read_csv('example_revenue.csv', parse_dates=['Date'])
df1.head()

,Date,Revenue
0,2018-01-01,0.00
1,2018-01-02,0.00
2,2018-01-03,3087464.68
3,2018-01-04,2770763.36
4,2018-01-05,0.00


In [38]:
df2 = pd.read_csv('example_weather.csv', parse_dates=[['dt', 'time']])
df2.rename(columns={'dt_time': "date"}, inplace=True)
df2.head()

,date,temperature,dew_point,humidity,wind,wind_speed,pressure,precipitation,condition
0,2018-01-01 00:20:00,0,0,1.0,S,11,982,0,Mist
1,2018-01-01 00:50:00,0,0,1.0,S,14,983,0,Mist
2,2018-01-01 01:20:00,0,0,1.0,S,11,982,0,Mist
3,2018-01-01 01:50:00,0,0,1.0,S,14,982,0,Mostly Cloudy
4,2018-01-01 02:20:00,0,0,1.0,S,14,981,0,Light Rain


### Drop Precipitation

In [39]:
# drop precipitation because it is constant
df2 = df2.drop(columns=['precipitation'])

### Downsampling

In [40]:
# because we need daily predictions, minutes data -> daily (downsampling)

downsampling_type = ({'temperature':'mean',
                      'dew_point':'mean',
                      'humidity':'mean',
                      'wind':'max',
                      'wind_speed':'mean',
                      'pressure':'mean',
                      'condition':'max'})
df2 = df2.resample('D', on='date').agg(downsampling_type)
df2.reset_index(inplace=True)

### Wind to numeric

In [41]:
# transform categoriacla features into numerical ones
wind_transformation = {'N':0, 'NNE':22.5,"NE":45,"ENE":67.5,
                       'E':90,'ESE':112.5, 'SE':135,'SSE':157.5,
                       'S':180,'SSW':202.5, 'SW':225,'WSW':247.5,
                       'W':270,'WNW':292.5,'NW':315,'NNW':337.5,
                       'N':0,'VAR':0, np.nan:0}

df2['wind'] = df2['wind'].replace(wind_transformation)

### One-hot encode condition


In [43]:
one_hot = pd.get_dummies(df2['condition'])
df2 = df2.drop('condition',axis = 1)
df2 = df2.join(one_hot)

df2.head()

,date,temperature,dew_point,humidity,wind,wind_speed,pressure,Mist,Mostly Cloudy,Partly Cloudy,Rain,Snow,Wintry Mix,Wintry Mix / Windy
0,2018-01-01,3.240741,3.018519,0.984444,225.0,20.555556,979.111111,0,0,0,0,0,1,0
1,2018-01-02,2.333333,2.175439,0.980351,225.0,13.543860,985.035088,0,1,0,0,0,0,0
2,2018-01-03,1.923077,1.653846,0.973077,202.5,15.461538,979.807692,0,0,1,0,0,0,0
3,2018-01-04,2.285714,1.428571,0.928571,157.5,24.040816,973.959184,0,1,0,0,0,0,0
4,2018-01-05,2.290909,1.618182,0.915818,202.5,23.563636,975.309091,0,0,0,1,0,0,0


### Interpolate missing values

In [44]:
# interpolate all except date
for col in df2.columns[1:]:
  df2[col] = df2[col].interpolate()

### Seasonality identification

In [47]:
core_columns =  [
    'temperature', 'dew_point', 'humidity', 
    'wind_speed', 'pressure'
]

for column in core_columns:
    decomp = seasonal_decompose(df2[column], model='additive', period=15, extrapolate_trend='freq')
# Add additional columns to dataframe
    df2[f"{column}_trend"] = decomp.trend
    df2[f"{column}_seasonal"] = decomp.seasonal

### Combine, Adjust &  Export Data

In [65]:
# Append 'Revenue' column to weather dataframe
df2['revenue'] = df1['Revenue']

In [72]:
df = pd.DataFrame(index=range(30), columns = ['date', 'temperature', 'dew_point', 'humidity', 'wind', 'wind_speed',
       'pressure', 'Cloudy', 'Cloudy / Windy', 'Fair', 'Fair / Windy', 'Fog',
       'Haze', 'Light Drizzle', 'Light Rain', 'Light Rain Shower',
       'Light Snow', 'Mist', 'Mostly Cloudy', 'Mostly Cloudy / Windy',
       'Partial Fog', 'Partly Cloudy', 'Partly Cloudy / Windy',
       'Patches of Fog', 'Rain', 'Rain / Windy', 'Rain Shower',
       'Rain Shower / Windy', 'Shallow Fog', 'Showers in the Vicinity',
       'Smoke', 'Snow', 'Snow / Windy', 'Snow Shower', 'Snow Shower / Windy',
       'T-Storm', 'T-Storm / Windy', 'Thunder', 'Thunder / Windy',
       'Thunder in the Vicinity', 'Wintry Mix', 'Wintry Mix / Windy',
       'temperature_trend', 'temperature_seasonal', 'dew_point_trend',
       'dew_point_seasonal', 'humidity_trend', 'humidity_seasonal',
       'wind_speed_trend', 'wind_speed_seasonal', 'pressure_trend',
       'pressure_seasonal', 'revenue']).fillna(0)


for col in df2.columns:
    df[col] = df2[col]

df.head()

,date,temperature,dew_point,humidity,wind,wind_speed,pressure,Cloudy,Cloudy / Windy,Fair,...,temperature_seasonal,dew_point_trend,dew_point_seasonal,humidity_trend,humidity_seasonal,wind_speed_trend,wind_speed_seasonal,pressure_trend,pressure_seasonal,revenue
0,2018-01-01,3.240741,3.018519,0.984444,225.0,20.555556,979.111111,0,0,0,...,-1.055212,-3.462649,-2.553531,0.876792,-0.080652,13.542524,10.253180,999.510569,-11.690120,0.00
1,2018-01-02,2.333333,2.175439,0.980351,225.0,13.543860,985.035088,0,0,0,...,0.084727,-3.653297,0.416936,0.876866,0.020149,13.593798,0.253530,999.217213,-17.488847,0.00
2,2018-01-03,1.923077,1.653846,0.973077,202.5,15.461538,979.807692,0,0,0,...,1.506660,-3.843945,2.382481,0.876940,0.055546,13.645073,-0.925201,998.923857,-16.940063,3087464.68
3,2018-01-04,2.285714,1.428571,0.928571,157.5,24.040816,973.959184,0,0,0,...,2.046837,-4.034593,2.574678,0.877014,0.023225,13.696347,2.770140,998.630501,-19.766513,2770763.36
4,2018-01-05,2.290909,1.618182,0.915818,202.5,23.563636,975.309091,0,0,0,...,1.950724,-4.225241,2.553300,0.877087,0.034660,13.747621,0.030888,998.337145,-14.410614,0.00


In [73]:
df.to_json('test2.json', orient="index")

Now, run service in BentoML and get prediction using prepared data. Sample query:

`curl -X POST -H "Content-Type: application/json" -d @test2.json http://0.0.0.0:3000/predict30`